In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import hvplot.pandas
%matplotlib inline

In [6]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")


api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [12]:
# Set the ticker
ticker =  ['SPY','AAPL','PFE','TSLA','AA']

# Set timeframe to "1Day"
timeframe = "1Day"

# Set start and end datetimes of 3 years from Today
start_date = pd.Timestamp("2020-03-06'", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-03-13", tz="America/New_York").isoformat()

# Get 3 years worth of historical data for TSLA
ticker_data = api.get_bars(
    ticker,
    timeframe,
    start=start_date,
    end=end_date
).df

ticker_data

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2020-03-06 05:00:00+00:00,11.360,11.7700,10.8000,11.01,10566407,61530,11.198566,AA
2020-03-09 04:00:00+00:00,9.620,10.1099,8.6100,8.69,10033959,44189,9.291311,AA
2020-03-10 04:00:00+00:00,9.330,9.4200,8.5500,9.26,10358291,46964,8.939416,AA
2020-03-11 04:00:00+00:00,8.820,9.2300,8.3000,9.20,9451032,46143,8.553895,AA
2020-03-12 04:00:00+00:00,8.350,8.3500,7.2400,7.51,12676874,48319,7.692379,AA
...,...,...,...,...,...,...,...,...
2022-03-07 05:00:00+00:00,856.300,866.1400,804.5700,804.58,23581848,679982,831.316312,TSLA
2022-03-08 05:00:00+00:00,795.530,849.9900,782.1700,824.40,25682302,710562,821.517965,TSLA
2022-03-09 05:00:00+00:00,839.480,860.5600,832.0100,858.97,19015970,521312,850.715956,TSLA


In [13]:
df_portfolio = api.get_bars(
    ticker,
    timeframe,
    start = '2020-03-06',
    end = end_date
).df

# Seperate ticker symnbols
SPY = df_portfolio[df_portfolio['symbol']=='SPY'].drop('symbol', axis=1)
AAPL =df_portfolio[df_portfolio['symbol']=='AAPL'].drop ('symbol', axis=1)
PFE = df_portfolio[df_portfolio['symbol']=='PFE'].drop ('symbol', axis=1)
TSLA = df_portfolio[df_portfolio['symbol']=='TSLA'].drop ('symbol', axis=1)
AA = df_portfolio[df_portfolio['symbol']=='AA'].drop ('symbol', axis=1)

# Combine Data
df_combined = pd.concat([SPY,AAPL,PFE,TSLA,AA], axis=1, keys =['SPY','AAPL','PFE','TSLA','AA'])
df_combined

SPY                                       \
                             open      high     low   close     volume   
timestamp                                                                
2020-03-06 05:00:00+00:00  293.15  298.7800  290.23  297.43  228918131   
2020-03-09 04:00:00+00:00  275.30  284.1900  273.45  276.32  309425747   
2020-03-10 04:00:00+00:00  284.64  288.5200  273.50  288.41  276444058   
2020-03-11 04:00:00+00:00  280.70  281.9400  270.88  274.25  256678789   
2020-03-12 04:00:00+00:00  256.00  266.6600  247.68  255.24  392221671   
...                           ...       ...     ...     ...        ...   
2022-03-07 05:00:00+00:00  431.55  432.3018  419.36  419.43  131006026   
2022-03-08 05:00:00+00:00  419.60  427.2100  415.12  416.25  158889879   
2022-03-09 05:00:00+00:00  425.14  429.5100  422.83  427.41  110176599   
2022-03-10 05:00:00+00:00  422.52  426.4300  420.44  425.48   91933614   
2022-03-11 05:00:00+00:00  428.12  428.7700  419.53  420.07   90832525   

                                                      AAPL                  \
                          trade_count        vwap     open    high     low   
timestamp                                                                    
2020-03-06 05:00:00+00:00     1642906  295.257537  282.000  290.82  281.23   
2020-03-09 04:00:00+00:00     2577867  280.319722  263.750  278.09  263.00   
2020-03-10 04:00:00+00:00     2516868  281.131339  277.140  286.44  269.37   
2020-03-11 04:00:00+00:00     2152461  277.627436  277.390  281.22  271.86   
2020-03-12 04:00:00+00:00     3709928  255.696186  255.940  270.00  248.00   
...                               ...         ...      ...     ...     ...   
2022-03-07 05:00:00+00:00     1390458  424.070071  163.360  165.02  159.04   
2022-03-08 05:00:00+00:00     1864070  419.755056  158.820  162.88  155.80   
2022-03-09 05:00:00+00:00     1079759  426.139268  161.475  163.41  159.41   
2022-03-10 05:00:00+00:00      891240  423.871053  160.200  160.39  155.98   
2022-03-11 05:00:00+00:00      809135  424.040518  158.930  159.28  154.50   

                           ...      TSLA                             AA  \
                           ...    volume trade_count        vwap   open   
timestamp                  ...                                            
2020-03-06 05:00:00+00:00  ...  12661418      259527  695.191900  11.36   
2020-03-09 04:00:00+00:00  ...  17077752      341256  627.386573   9.62   
2020-03-10 04:00:00+00:00  ...  15594488      302024  638.495306   9.33   
2020-03-11 04:00:00+00:00  ...  13413587      246538  634.335750   8.82   
2020-03-12 04:00:00+00:00  ...  18922102      371893  564.484197   8.35   
...                        ...       ...         ...         ...    ...   
2022-03-07 05:00:00+00:00  ...  23581848      679982  831.316312  89.56   
2022-03-08 05:00:00+00:00  ...  25682302      710562  821.517965  82.00   
2022-03-09 05:00:00+00:00  ...  19015970      521312  850.715956  79.80   
2022-03-10 05:00:00+00:00  ...  19210350      559271  830.235671  83.69   
2022-03-11 05:00:00+00:00  ...  21892445      731665  808.992306  80.49   

                                                                        \
                              high    low  close    volume trade_count   
timestamp                                                                
2020-03-06 05:00:00+00:00  11.7700  10.80  11.01  10566407       61530   
2020-03-09 04:00:00+00:00  10.1099   8.61   8.69  10033959       44189   
2020-03-10 04:00:00+00:00   9.4200   8.55   9.26  10358291       46964   
2020-03-11 04:00:00+00:00   9.2300   8.30   9.20   9451032       46143   
2020-03-12 04:00:00+00:00   8.3500   7.24   7.51  12676874       48319   
...                            ...    ...    ...       ...         ...   
2022-03-07 05:00:00+00:00  92.3200  83.51  85.40  12456291      127982   
2022-03-08 05:00:00+00:00  85.5999  77.77  82.37  12536226      121892   
2022-03-09 05:00:00+00:00  80.8100 

In [14]:
# Fetch the closing price each day
spy_price = (df_combined['SPY']['close'])
aapl_price = (df_combined['AAPL']['close'])
pfe_price = (df_combined['PFE']['close'])
tsla_price = (df_combined['TSLA']['close'])
aa_price = (df_combined['AA']['close'])
aa_price

timestamp
2020-03-06 05:00:00+00:00    11.01
2020-03-09 04:00:00+00:00     8.69
2020-03-10 04:00:00+00:00     9.26
2020-03-11 04:00:00+00:00     9.20
2020-03-12 04:00:00+00:00     7.51
                             ...  
2022-03-07 05:00:00+00:00    85.40
2022-03-08 05:00:00+00:00    82.37
2022-03-09 05:00:00+00:00    79.52
2022-03-10 05:00:00+00:00    82.94
2022-03-11 05:00:00+00:00    79.04
Name: close, Length: 509, dtype: float64

In [15]:
df_close = pd.concat([spy_price,aapl_price,pfe_price,tsla_price,aa_price], axis=1, keys =['SPY','AAPL','PFE','TSLA','AA'])

df_close.index = df_close.index.date

df_dreturns = df_close.pct_change().dropna()

df_dreturns

,SPY,AAPL,PFE,TSLA,AA
2020-03-09,-0.070975,-0.078837,-0.050828,-0.135725,-0.210718
2020-03-10,0.043754,0.072022,0.040614,0.061398,0.065593
2020-03-11,-0.049097,-0.034730,-0.069962,-0.017201,-0.006479
2020-03-12,-0.069316,-0.098755,-0.066832,-0.116172,-0.183696
2020-03-13,0.058612,0.119808,0.089607,-0.024851,0.065246
...,...,...,...,...,...
2022-03-07,-0.029479,-0.023718,-0.013772,-0.040213,-0.058019
2022-03-08,-0.007582,-0.011676,-0.011255,0.024634,-0.035480
2022-03-09,0.026811,0.034997,0.027614,0.041934,-0.034600
2022-03-10,-0.004516,-0.027186,0.009231,-0.024064,0.043008


In [ ]:
df = pandas.DataFrame(randn(4,4))
df['AAL'].max()
df['AAP'].min()

In [21]:
df_close.max()

SPY      477.71
AAPL     506.09
PFE       61.25
TSLA    2238.75
AA        90.66
dtype: float64

In [22]:
df_close.min()

SPY     222.68
AAPL    106.84
PFE      28.49
TSLA    330.21
AA        5.48
dtype: float64

In [16]:
df_close.hvplot.line()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [ ]:
 Which industry was affected the most by Covid?
    * Pre vs Post
    # Accroding to the plot shown above, we can conclude that both Apple(AAPL) and Tesla(TSLA) were the most affected by Covid since they could all be recognized as high tech-manufacture industry.
    # Plus, their fuse moment were the same.

In [ ]:
* Which Stock performed the best after Covid?
   # AA performed the best after Covid($44.58 01/03/2023 compareed to $11.01 03/06/2020, 404.9% growth)

In [ ]:
* Which stock performed the worst after Covid?
   # Despite the stock split of TSLA affected its value per stock(1 -> 3 stocks), TSLA and AAPL

In [ ]:
 * What were the highest and lowest prices of each stock throughout the past 2 years
    # MAX:
SPY      477.71
AAPL     506.09
PFE       61.25
TSLA    2238.75
AA        90.66
    # MIN:
SPY     222.68
AAPL    106.84
PFE      28.49
TSLA    330.21
AA        5.48
dtype: float64

In [ ]:
* What was the overall trend of each industry after Covid?
